In [42]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import json
import import_ipynb
import tqdm


In [12]:
from torch.utils.data import TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader # 데이터로더
from torch.utils.data import Dataset
from torchtext.vocab import GloVe
from torchtext.legacy import data, datasets
from layers import *


In [29]:
with open('./squad1.1/dev-v1.1.json') as f: 
        stuff = json.load(f)
        
t_array = stuff.get("data")
context_list, questions_list, answers_list = [],[],[]
print(json.dumps(stuff["data"][0], indent=2))

{
  "title": "Super_Bowl_50",
  "paragraphs": [
    {
      "context": "Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24\u201310 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the \"golden anniversary\" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as \"Super Bowl L\"), so that the logo could prominently feature the Arabic numerals 50.",
      "qas": [
        {
          "answers": [
            {
              "answer_start": 177,
              "text": "Denver Broncos"
     

In [66]:
#torchtext.datasets.SQuAD1(root: str = '.data', split: Union[Tuple[str], str] = ('train', 'dev'))
#하다 이거 가져오자

def help_index(context_list, offset):
    p = 0
    index =-1
    for tokens in context_list:
        index = index + 1
        for string in tokens:
            if p == offset:
                break
            p = p +len(string) + 1
        
    return index


def parsing(path):
    with open(path) as f: 
        stuff = json.load(f)
        
    t_array = stuff.get("data")
    context_list, questions_list, answers_list = [],[],[]
    #questions_list = (context#, query#, query tokens)
    #answers_list = (context#, num, answer_start, 정답 text)
    
    
    #print(json.dumps(train_stuff["data"][0], indent=2))
    for c in t_array:
        o =c.get("paragraphs") #o는 list
        for n in o:
            context_list.append(n.get("context").split())

    for c in t_array:
        o = c.get("paragraphs")
        idx = -1
        for n in o:
            t = n.get("qas")
            idx=idx+1
            num=-1
            for k in t:
                num=num+1
                questions_list.append((idx, num,k.get("question").split())) #id 언제 쓸까?? 필요없는것 같은데 걍 버려보자
                

    for c in t_array:
        o = c.get("paragraphs")
        idx=-1
        for n in o:
            t = n.get("qas")
            idx=idx+1
            num =-1
            for k in t:
                p=k.get("answers")
                for j in p:
                    num= num+1
                    trans = help_index(context_list[idx], j.get("answer_start"))
                    #answers_list.append((idx, num, j.get("answer_start"), j.get("text").split()))
                    answers_list.append((idx, num,trans,  j.get("text").split()))
    #print(context_list[0]) #print(questions_list[:12])  #print(answers_list[:12])   #print(len(context_list)) #print(questions_list[1][2])
    print(answers_list[:3])
    print(context_list[0])
    print(len(context_list[0]))
    
    data_form = []
    k = -1
    for num in range(len(questions_list)):
        if questions_list[num][1]==0 and  k <= len(context_list):
            k=k+1
        p=(context_list[k], questions_list[num][2], (answers_list[num][1], len(answers_list[num][3])))
        data_form.append(p)
    #data_form  :=  (context 토큰들, questoin 토큰들, answer start index, answer 토큰 길이)
    
    new_vocabs=[]
    for n in range(len(data_form)):
        new_vocabs.append(data_form[n][1])
        new_vocabs.append(answers_list[n][3]) 

    new_vocabs= new_vocabs+ context_list
    
    
    return new_vocabs
   

#TEXT1은 context안에 있는 모든 토큰들을 glove와 matchign 후 indexing, 후 embedding table.
#print('단어 집합의 크기 : {}'.format(len(TEXT1.vocab)))
#print(TEXT1.vocab.stoi['this'])
#print(TEXT1.vocab.vectors[90]) # <unk>의 임베딩 벡터값
#embedding_layer(torch.LongTensor([39])) 
#정리하면 TEXT1 vocab에 각 context 단어별로 indexing이 되어있음 즉 단어:indexing 
#그리고 embedding_layer에는 각 인덱싱 number별로 100dimension 짜리 vector가 있음.
#근데 꼭 torchtext로 이렇게 단어장을 만들어야 할 필요가 있을까?,, 모르겟다.
#나중에 nn.Embedding 하면 이값으로 초기화됨 ㅎㅎ 
          


In [67]:
dev_vocabs = parsing('./squad1.1/dev-v1.1.json')


[(0, 0, 123, ['Denver', 'Broncos']), (0, 1, 123, ['Denver', 'Broncos']), (0, 2, 123, ['Denver', 'Broncos'])]
['Super', 'Bowl', '50', 'was', 'an', 'American', 'football', 'game', 'to', 'determine', 'the', 'champion', 'of', 'the', 'National', 'Football', 'League', '(NFL)', 'for', 'the', '2015', 'season.', 'The', 'American', 'Football', 'Conference', '(AFC)', 'champion', 'Denver', 'Broncos', 'defeated', 'the', 'National', 'Football', 'Conference', '(NFC)', 'champion', 'Carolina', 'Panthers', '24–10', 'to', 'earn', 'their', 'third', 'Super', 'Bowl', 'title.', 'The', 'game', 'was', 'played', 'on', 'February', '7,', '2016,', 'at', "Levi's", 'Stadium', 'in', 'the', 'San', 'Francisco', 'Bay', 'Area', 'at', 'Santa', 'Clara,', 'California.', 'As', 'this', 'was', 'the', '50th', 'Super', 'Bowl,', 'the', 'league', 'emphasized', 'the', '"golden', 'anniversary"', 'with', 'various', 'gold-themed', 'initiatives,', 'as', 'well', 'as', 'temporarily', 'suspending', 'the', 'tradition', 'of', 'naming', 'eac

In [15]:
'''
from torchtext.legacy.data import Iterator

batch_size = 5

from torch.utils.data import DataLoader

train_dataloader = DataLoader(context_list, batch_size=64, shuffle=True)

train_loader = Iterator(dataset = context_list, batch_size = batch_size)
#q_a에서는 다른 형식이필요할듯
'''

#hidden state size = 100 ,, 이게 cnn만 해당인걸까?? 아니면 lstm 도 해당일까
#lstm 은 단어의 dimension을 받겠지. 100이 아닐듯

'\nfrom torchtext.legacy.data import Iterator\n\nbatch_size = 5\n\nfrom torch.utils.data import DataLoader\n\ntrain_dataloader = DataLoader(context_list, batch_size=64, shuffle=True)\n\ntrain_loader = Iterator(dataset = context_list, batch_size = batch_size)\n#q_a에서는 다른 형식이필요할듯\n'

In [40]:
train_vocabs = parsing('./squad1.1/train-v1.1.json')
dev_vocabs = parsing('./squad1.1/dev-v1.1.json')

#train set
TEXT1 = data.Field(sequential=True, batch_first=True, lower=True)
LABEL1 = data.Field(sequential=False, batch_first=True)
TEXT1.build_vocab(train_vocabs, vectors=GloVe(name='6B', dim=100))
LABEL1.build_vocab(train_vocabs)
embedding_layer1 = nn.Embedding.from_pretrained(TEXT1.vocab.vectors, freeze=False)


#dev set
TEXT2 = data.Field(sequential=True, batch_first=True, lower=True)
LABEL2 = data.Field(sequential=False, batch_first=True)
TEXT2.build_vocab(dev_vocabs, vectors=GloVe(name='6B', dim=100))
LABEL2.build_vocab(dev_vocabs)
embedding_layer2 = nn.Embedding.from_pretrained(TEXT2.vocab.vectors, freeze=False)


In [68]:
class Squad(Dataset):
    def __init__(self,pdata):
        self.data = pdata
  # 총 데이터의 개수를 리턴
    def __len__(self): 
        return len(self.data)
    def __getitem__(self, idx):
        return self.data[idx][0], self.data[idx][1], self.data[idx][2]   #context, query, answer
# 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
#tensor는 string form이 없음,,,, 만약에 그냥 토큰들을 리턴하면... tensor형태로 반환을 못하는데?
  

In [70]:
learning_rate = 0.5
batch_size = 60
epochs = 12
dropout_rate = 0.2

#나중에 넣게 될 폼
#train(model 아마 BIDAF?, train_vocabs, dev_vocabs, learning_rate, epochs)

#model = layers.BIDAF
def train(model, train_data, dev_data, learning_rate, epochs):
    train, dev = Squad(train_data), Squad(dev_data)
    
    train_dataloader = DataLoader(train, batch_size=batch_size, shuffle=False)
    val_dataloader = DataLoader(val, batch_size=batch_size)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adadelta(model.parameters(),lr= learning_rate)
    
    
    if torch.cuda.is_available():
        model = model.cuda()
        criterion.cuda()
    
    best_acc = 0 #이거뭐지...
    
    model.train()
    for epoch_num in range(epochs):
        total_acc_train = 0
        total_loss_train = 0

        # train_input 확인
        for train_context, train_query, train_answer in tqdm(train_dataloader):
            train_context = train_context.to(device)  #device에 할당
            train_query = train_query.to(device)
            train_answer = train_answer.to(device)

            model.zero_grad()

            start, end = model(train_context, train_query)

            batch_loss1 = criterion(start, train_answer[0]) #이런 느낌
            batch_loss2 = criterion(end, train_answer[1])
            total_loss_train += (batch_loss1.item() + batch_loss2.item())/batch_size

            total_loss_train.backward()
            #batch_loss.backward()
            optimizer.step()

        total_acc_val = 0
        total_loss_val = 0
        
    model.eval()
    with torch.no_grad():


In [39]:
print(TEXT1.vocab.stoi['this'])
print(TEXT1.vocab.vectors[39])
print(TEXT2.vocab.stoi['this'])

print(embedding_layer1(torch.LongTensor([50])))
print(embedding_layer2(torch.LongTensor([46])))


print('임베딩 벡터의 개수와 차원 : {} '.format(TEXT1.vocab.vectors.shape))
print('임베딩 벡터의 개수와 차원 : {} '.format(TEXT2.vocab.vectors.shape))


50
tensor([-0.0201,  0.0375,  0.3536,  0.1658,  0.0948,  0.0875,  0.7950,  0.2076,
        -0.5813, -0.0771,  0.1966, -0.4031, -0.0782,  0.2917,  0.1698,  0.0039,
         0.7365,  0.3902, -0.0973, -0.1731,  0.6394, -0.2815,  0.1162,  0.6753,
         0.6317, -0.7825, -0.0523, -0.2793,  0.4608,  0.1246, -0.7226,  0.7339,
         0.1216,  0.2347,  0.0371,  0.1266, -0.5684,  0.6287, -0.2241,  0.1152,
        -0.5707,  0.1804,  0.4233,  0.0414,  0.3678,  0.2629,  0.4715, -0.8360,
         0.3344, -0.7268, -0.5073,  0.0337,  0.3284,  0.8632, -0.4569, -3.1522,
        -0.5397,  0.0334,  1.5954,  0.7581, -0.5242,  0.6155, -0.2672,  0.4432,
         0.9508, -0.2994, -0.0041,  0.3970,  0.0221, -0.1285, -0.1169,  0.1899,
        -0.1981, -0.4304, -0.0754,  0.4359, -0.6272, -0.2513, -1.2143, -0.4029,
         0.5143,  0.1677, -0.3184,  0.3868, -1.4578, -0.4342,  0.3969, -0.5009,
         0.4859, -0.4023,  0.0056,  0.0370, -0.1912,  0.3176, -0.7425, -0.0972,
        -0.1980,  0.0626,  0.2839, -0

In [ ]:
def train_loop(dataloader, loss_fn, optimizer):#model은 안에서 정의하자
    
    